# Laboratorio 5:
## Aplicación del problema de la Dieta al ganado

**Motivación:** El ganado de raza Brangus durante la fase de engorde requiere un balance preciso de nutrientes para garantizar que la carne obtenida mantenga una adecuada cobertura de grasa y marmoleo para el control de calidad. Además, se desea prevenir trastornos como acidosis ruminal o sobreengrasamiento que afectan la calidad de la carne. 

La fase de engorde suelde durar entre 60 y 120 días. Esta fase es muy importante para controlar la calidad incrementando ciertos alimentos a la dieta del ganado, mientras se disminuye el forraje debido a que durante esta fase empeora la eficiencia y velocidad del engorde. Sin embargo, debe mantenerse un nivel mínimo de consumo para que puedan mantener ciertas funciones digestivas y de producción de leche.

**Problema:** Supondremos que contamos con un conjunto de alimentos $F$ y un conjunto de nutrientes $N$ que son requeridos para la dieta del ganado. Para cada alimento $i \in F$, $a_{ij}$ representa la cantidad del nutriente $j \in N$ contenida en cada porción del alimento, $p_i$ es el peso por porción y $c_i$ es el costo por cada porción.

Debe seleccionarse el número de porciones de cada alimento que se deben incluir en la dieta diaria del ganado, de tal forma que se satisfagan las siguientes restricciones:
* Los niveles totales de cada nutriente $j \in N$ contenidos en la dieta, deben estar dentro de un rango $[\text{Nmin}_j; \text{Nmax}_j]$, para garantizar una dieta de engorde saludable. 
* El peso total de la dieta diaria no debe exceder el límite máximo $P_{\max}$.

No es posible seleccionar fracciones de una porción para ningún alimento.

Se busca formular un modelo de programación lineal entera que permita encontrar una dieta diaria que satisfaga los requerimientos nutricionales y de peso, y que tenga el menor costo posible.

Para ello, consideremos los siguientes conjuntos y parámetros.

**Conjuntos y Parámetros:**

- $F$ es el conjunto de alimentos

- $N$ es el conjunto de nutrientes

- $c_i$, con $i \in F$, es el costo unitario del alimento $i$.  

- $a_{ij}$, con $i \in F,\; j \in N$ es la cantidad del nutriente $j$ presente en una porción del alimento $i$.

- $\text{Nmin}_j$, con $j \in N$, es el nivel mímino del nutriente $j$. 

- $\text{Nmax}_j$, con $j \in N$, es el máximo nivel del nutriente $j$.

- $p_i$, con $i \in F$ es el peso del alimento $i$ por porción.

- $P_{\max}$ es el peso máximo que debe tener un platillo.

Consideremos las siguientes **variables de decisión**:

- $x_i \in \mathbb{Z}_+$, con $i \in F$, es el número de porciones del alimento $i$ que se deben incluir en la dieta del ganado. Observar que el número de porciones debe ser entero y no negativo.

La **función objetivo** consiste en minimizar el costo total de las porciones de cada alimento seleccionado:
$$
\min \sum_{i \in F} c_i x_i
$$

**Restricciones:**

Se deben respetar los requerimientos (mínimos y máximos) de cada nutriente $j \in N$:

\begin{align*}
\sum_{i \in F} a_{ij} x_i \geq \text{Nmin}_j, \qquad \forall j \in N \\
\sum_{i \in F} a_{ij} x_i \leq \text{Nmax}_j, \qquad \forall j \in N 
\end{align*}

Cada comida no debe sobrepasar el peso máximo permitido de $P_{\max}$ gramos:

$$
\sum_{i \in F} p_i x_i \leq P_{\max}.
$$


### Instancia:

Consideremos la siguiente instancia de prueba.

Nota: Los nutrientes son medidos en gramos, excepto la energía que se mida en megacalorías.

In [1]:
# Información de los alimentos: costo (USD/kg MS) y peso por porción (kg)
alimentos = {
    # Concentrados energéticos y proteicos
    'Maíz': (0.35, 0.5),              
    'Harina de Soya': (0.55, 0.4),    
    'Cebada': (0.30, 0.5),
    'Avena': (0.40, 0.5),
    'Trigo': (0.45, 0.4),
    'Melaza': (0.60, 0.3),            
    'Harina de Pescado': (1.50, 0.1), 
    'Torta de Algodón': (0.85, 0.3),
    'Pulpa de Remolacha': (0.25, 0.6),
    'Gluten Feed de Maíz': (0.40, 0.4),
    'DDGS': (0.38, 0.4),
    'Salvado de Trigo': (0.32, 0.4),

    # Forrajes y subproductos fibrosos
    'Heno de Alfalfa': (0.20, 1.5),
    'Heno de Rye Grass': (0.18, 1.2),
    'Heno de Avena': (0.22, 1.2),
    'Ensila de Maíz': (0.15, 2.0),
    'Paja de Trigo': (0.10, 1.5),
    'Cascarilla de Soya': (0.28, 0.8),
    'Pulpa de Remolacha': (0.25, 0.6),

    # Suplementos
    'Aceite Vegetal': (1.20, 0.05),
    'Carbonato de Calcio': (0.12, 0.05)
}


# Requerimientos nutricionales (mínimo, máximo) para dieta de engorde (base MS)
nutrientes = {
    'Proteína': (650, 1800),   # g/kg de dieta
    'Energía': (2.2, 10),    # Mcal/kg MS
    'Fibra': (150, 1500),      # g/kg
    'Grasa': (15, 180),        # g/kg
    'Calcio': (3, 30),        # g/kg
    'Fósforo': (2, 17)         # g/kg
}

# Peso máximo permitido de la dieta (kg de MS)
Vmax = 6

# Contenido de nutrientes (por kg de materia seca)
a = {
    'Maíz': {'Proteína': 90, 'Energía': 3.4, 'Fibra': 120, 'Grasa': 40, 'Calcio': 0.3, 'Fósforo': 2.7},
    'Harina de Soya': {'Proteína': 440, 'Energía': 3.0, 'Fibra': 60, 'Grasa': 18, 'Calcio': 2.5, 'Fósforo': 6.5},
    'Heno de Alfalfa': {'Proteína': 180, 'Energía': 2.2, 'Fibra': 400, 'Grasa': 25, 'Calcio': 15.0, 'Fósforo': 2.5},
    'Cebada': {'Proteína': 120, 'Energía': 3.1, 'Fibra': 180, 'Grasa': 25, 'Calcio': 0.8, 'Fósforo': 3.8},
    'Avena': {'Proteína': 110, 'Energía': 2.9, 'Fibra': 200, 'Grasa': 45, 'Calcio': 0.5, 'Fósforo': 3.5},
    'Trigo': {'Proteína': 130, 'Energía': 3.3, 'Fibra': 130, 'Grasa': 18, 'Calcio': 0.6, 'Fósforo': 3.8},
    'Melaza': {'Proteína': 50, 'Energía': 2.7, 'Fibra': 50, 'Grasa': 5, 'Calcio': 9.0, 'Fósforo': 0.6},
    'Harina de Pescado': {'Proteína': 600, 'Energía': 4.5, 'Fibra': 0, 'Grasa': 80, 'Calcio': 50.0, 'Fósforo': 28.0},
    'Torta de Algodón': {'Proteína': 370, 'Energía': 3.1, 'Fibra': 150, 'Grasa': 120, 'Calcio': 2.5, 'Fósforo': 6.5},
    'Pulpa de Remolacha': {'Proteína': 90, 'Energía': 2.8, 'Fibra': 350, 'Grasa': 10, 'Calcio': 8.0, 'Fósforo': 1.0},
    'Ensila de Maíz': {'Proteína': 80, 'Energía': 2.6, 'Fibra': 300, 'Grasa': 25, 'Calcio': 3.0, 'Fósforo': 2.0},
    'Paja de Trigo': {'Proteína': 40, 'Energía': 1.8, 'Fibra': 450, 'Grasa': 10, 'Calcio': 3.5, 'Fósforo': 1.0},
    'Heno de Rye Grass': {'Proteína': 160, 'Energía': 2.3, 'Fibra': 380, 'Grasa': 20, 'Calcio': 4.0, 'Fósforo': 2.5},
    'Heno de Avena': {'Proteína': 120, 'Energía': 2.4, 'Fibra': 400, 'Grasa': 25, 'Calcio': 3.0, 'Fósforo': 2.0},
    'Cascarilla de Soya': {'Proteína': 120, 'Energía': 3.0, 'Fibra': 320, 'Grasa': 20, 'Calcio': 2.5, 'Fósforo': 2.0},
    'Salvado de Trigo': {'Proteína': 160, 'Energía': 2.9, 'Fibra': 120, 'Grasa': 45, 'Calcio': 1.3, 'Fósforo': 11.0},
    'Gluten Feed de Maíz': {'Proteína': 220, 'Energía': 3.1, 'Fibra': 100, 'Grasa': 45, 'Calcio': 0.7, 'Fósforo': 9.0},
    'DDGS': {'Proteína': 300, 'Energía': 3.4, 'Fibra': 100, 'Grasa': 100, 'Calcio': 2.0, 'Fósforo': 8.0},
    'Aceite Vegetal': {'Proteína': 0, 'Energía': 8.5, 'Fibra': 0, 'Grasa': 999, 'Calcio': 0, 'Fósforo': 0},
    'Carbonato de Calcio': {'Proteína': 0, 'Energía': 0, 'Fibra': 0, 'Grasa': 0, 'Calcio': 380.0, 'Fósforo': 0},
}


Para implementar la siguiente instancia es necesario realizar un preprocesamiento de datos:

In [2]:
# Importamos los paquetes
import gurobipy as gp
from gurobipy import GRB

# Indices de los conjuntos de alimentos y nutrientes
idx_F = dict(zip(alimentos.keys(),range(1,len(alimentos)+1)))
idx_N = dict(zip(nutrientes.keys(),range(1,len(nutrientes)+1)))

# Parámetros y conjuntos
F,c,p = gp.multidict({idx_F[i]:alimentos[i] for i in idx_F})
N,N_min,N_max = gp.multidict({idx_N[j]:nutrientes[j] for j in idx_N})
a = gp.tupledict({(idx_F[i],idx_N[j]):a[i][j]*p[idx_F[i]] for i in idx_F for j in idx_N})


A continuación implementaremos el modelo usando la interfaz de gurobi en Python.

In [3]:
try:
    m = gp.Model("Problema de la Dieta")
    x = m.addVars(F,vtype=GRB.INTEGER, name="x")
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)
    m.addConstrs((gp.quicksum(a[i,j]*x[i] for i in F) >= N_min[j] for j in N),'Rest_1')
    m.addConstrs((gp.quicksum(a[i,j]*x[i] for i in F) <= N_max[j] for j in N),'Rest_2')
    m.addConstr(x.prod(p,'*') <= Vmax, "Rest_3")
    m.optimize()
    # Guardamos la solución
    sol_opt = {i:[next((k for k in idx_F if idx_F[k] == i), None),p[i],x[i].x,c[i]] for i in F if x[i].x >= 1e-6}
    z_opt = round(m.objVal,2)
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))
except AttributeError:
    print('Encountered an attribute error')

Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-07
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.1.0 25B78)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 20 columns and 240 nonzeros
Model fingerprint: 0xac86f92c
Variable types: 0 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-02, 7e+02]
  Objective range  [1e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+03]
Presolve removed 5 rows and 1 columns
Presolve time: 0.00s
Presolved: 8 rows, 19 columns, 144 nonzeros
Variable types: 0 continuous, 19 integer (3 binary)
Found heuristic solution: objective 3.2000000
Found heuristic solution: objective 2.7000000

Root relaxation: objective 5.701700e-01, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Ob

Para visualizar los resultados mostraremos el menú seleccionado en una tabla.

In [4]:
# Mostramos el menú
import tabulate as tb
def menu(z_opt,sol_opt):
    print('+{:-<61}+'.format('-'))
    print('{:^63}'.format('Menú óptimo'))
    print('{:^63}'.format(str(z_opt)+' USD'))
    print('{:^63}'.format(f'{round(sum(sol_opt[i][1]*sol_opt[i][2] for i in sol_opt),2)} kg'))
    print(tb.tabulate(sol_opt.values(),headers=['Alimento','Peso','Porción','Precio por unidad'], tablefmt="pretty"))
menu(z_opt,sol_opt)


+-------------------------------------------------------------+
                          Menú óptimo                          
                           0.92 USD                            
                            4.0 kg                             
+-------------------+------+---------+-------------------+
|     Alimento      | Peso | Porción | Precio por unidad |
+-------------------+------+---------+-------------------+
|       DDGS        | 0.4  |   1.0   |       0.38        |
| Heno de Rye Grass | 1.2  |   3.0   |       0.18        |
+-------------------+------+---------+-------------------+


### Parte 1:

Debido a que durante la fase de engorde, el ganado debe consumir menos forrage, solo se puede consumir una porción de este tipo de alimentos. 

- Agregar una nueva restricción al modelo que permita limitar la cantidad de alimentos tipo forrage que se puede consumir. 
- Volver a resolver la instancia.
- Mostrar el nuevo menú.

In [5]:
# Lista de alimentos tipo forrajes
forraje = ['Heno de Alfalfa', 'Avena','Ensila de Maíz','Paja de Trigo','Heno de Rye Grass','Heno de Avena','Cascarilla de Soya']

### Solución:

### Parte 2:

Dado que mezclar diferentes tipos de forraje suele ser dañino para el ganado, es preferible seleccionar un único alimento de este tipo y como máximo una porción del alimento (si es seleccionado).

- Agregar una nueva restricción al modelo que permita limitar la variedad y consumo de alimentos tipo forrage que se puede consumir. 
- Volver a resolver la instancia.
- Mostrar el nuevo menú.

### Solución: